# <center> <img src="../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Big Data** </center>
---
### <center> **Spring 2025** </center>
---
### <center> **Examples on storage solutions for Big Data (files)** </center>
---
**Profesor**: Dr. Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

#### Creacion de la conexión con el cluster de spark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQL-Storage-Solutions") \
    .master("spark://078b2e28e517:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/11 00:41:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/11 00:41:10 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


### Car rental service

In [3]:
from team_name.spark_utils import SparkUtils
agencies_schema = SparkUtils.generate_schema([("agency_id", "string"), ("agency_info", "string")])

agencies_df = spark.read \
                .schema(agencies_schema) \
                .option("header", "true") \
                .csv("/home/jovyan/notebooks/data/rentals_dataset/agencies.csv")

agencies_df.printSchema()

agencies_df.show(5, truncate=False)

root
 |-- agency_id: string (nullable = true)
 |-- agency_info: string (nullable = true)



25/03/11 00:41:26 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/03/11 00:41:41 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


+---------+-----------------------------------------------------+
|agency_id|agency_info                                          |
+---------+-----------------------------------------------------+
|1        |{'agency_name': 'NYC Rentals', 'city': 'New York'}   |
|2        |{'agency_name': 'LA Car Rental', 'city': 'Londres'}  |
|3        |{'agency_name': 'Zapopan Auto', 'city': 'Zapopan'}   |
|4        |{'agency_name': 'SF Cars', 'city': 'San Francisco'}  |
|5        |{'agency_name': 'Mexico Cars', 'city': 'Mexico City'}|
+---------+-----------------------------------------------------+



In [4]:
from pyspark.sql.functions import get_json_object
agencies_df = agencies_df.withColumn("agency_name", get_json_object(agencies_df.agency_info, "$.agency_name"))
agencies_df.show(truncate=False)

+---------+-----------------------------------------------------+-------------+
|agency_id|agency_info                                          |agency_name  |
+---------+-----------------------------------------------------+-------------+
|1        |{'agency_name': 'NYC Rentals', 'city': 'New York'}   |NYC Rentals  |
|2        |{'agency_name': 'LA Car Rental', 'city': 'Londres'}  |LA Car Rental|
|3        |{'agency_name': 'Zapopan Auto', 'city': 'Zapopan'}   |Zapopan Auto |
|4        |{'agency_name': 'SF Cars', 'city': 'San Francisco'}  |SF Cars      |
|5        |{'agency_name': 'Mexico Cars', 'city': 'Mexico City'}|Mexico Cars  |
+---------+-----------------------------------------------------+-------------+



In [5]:
brands_schema = SparkUtils.generate_schema([("brand_id", "integer"), ("brand_info", "string")])
brands_df = spark.read.option("header", "true").schema(brands_schema).csv("/home/jovyan/notebooks/data/rentals_dataset/brands.csv")
brands_df.printSchema()
brands_df = brands_df.withColumn("brand_name", get_json_object(brands_df.brand_info, "$.brand_name"))
brands_df.show(5, truncate=False)

root
 |-- brand_id: integer (nullable = true)
 |-- brand_info: string (nullable = true)

+--------+------------------------------------------------------+-------------+
|brand_id|brand_info                                            |brand_name   |
+--------+------------------------------------------------------+-------------+
|1       |{'brand_name': 'Mercedes-Benz', 'country': 'Tanzania'}|Mercedes-Benz|
|2       |{'brand_name': 'BMW', 'country': 'Hungary'}           |BMW          |
|3       |{'brand_name': 'Audi', 'country': 'Senegal'}          |Audi         |
|4       |{'brand_name': 'Ford', 'country': 'Tuvalu'}           |Ford         |
|5       |{'brand_name': 'BYD', 'country': 'Italy'}             |BYD          |
+--------+------------------------------------------------------+-------------+
only showing top 5 rows



In [6]:
cars_schema = SparkUtils.generate_schema([("car_id", "integer"), ("car_info", "string")])
cars_df = spark.read.option("header", "true").schema(cars_schema).csv("/home/jovyan/notebooks/data/rentals_dataset/cars.csv")
cars_df.printSchema()
cars_df = cars_df.withColumn("car_name", get_json_object(cars_df.car_info, "$.car_name")) \
                .withColumn("brand_id", get_json_object(cars_df.car_info, "$.brand_id")) 
cars_df.show(5, truncate=False)

root
 |-- car_id: integer (nullable = true)
 |-- car_info: string (nullable = true)

+------+-------------------------------------------------------------------------------------+---------------------------------+--------+
|car_id|car_info                                                                             |car_name                         |brand_id|
+------+-------------------------------------------------------------------------------------+---------------------------------+--------+
|1     |{'car_name': 'Tucker, Hull and Gallegos Model 1', 'brand_id': 5, 'price_per_day': 68}|Tucker, Hull and Gallegos Model 1|5       |
|2     |{'car_name': 'Howard-Snow Model 7', 'brand_id': 5, 'price_per_day': 55}              |Howard-Snow Model 7              |5       |
|3     |{'car_name': 'Wagner LLC Model 2', 'brand_id': 2, 'price_per_day': 194}              |Wagner LLC Model 2               |2       |
|4     |{'car_name': 'Campos PLC Model 8', 'brand_id': 1, 'price_per_day': 107}        

In [7]:
customers_schema = SparkUtils.generate_schema([("customer_id", "integer"), ("customer_info", "string")])
customers_df = spark.read.option("header", "true").schema(customers_schema).csv("/home/jovyan/notebooks/data/rentals_dataset/customers.csv")
customers_df.printSchema()
customers_df = customers_df.withColumn("customer_name", get_json_object(customers_df.customer_info, "$.customer_name"))
customers_df.show(5, truncate=True)

root
 |-- customer_id: integer (nullable = true)
 |-- customer_info: string (nullable = true)

+-----------+--------------------+-------------------+
|customer_id|       customer_info|      customer_name|
+-----------+--------------------+-------------------+
|          1|{'customer_name':...|  Martin Graves DVM|
|          2|{'customer_name':...|   Frederick Wilson|
|          3|{'customer_name':...|       Gabriela Lee|
|          4|{'customer_name':...|     Devin Thornton|
|          5|{'customer_name':...|Christopher Simmons|
+-----------+--------------------+-------------------+
only showing top 5 rows



In [8]:
rental_cars_schema = SparkUtils.generate_schema([("rental_id", "integer"), ("rental_info", "string")])
rental_cars_df = spark.read.option("header", "true").schema(rental_cars_schema).csv("/home/jovyan/notebooks/data/rentals_dataset/rentals/")
rental_cars_df.printSchema()
rental_cars_df.show(5, truncate=False)

root
 |-- rental_id: integer (nullable = true)
 |-- rental_info: string (nullable = true)

+---------+--------------------------------------------------+
|rental_id|rental_info                                       |
+---------+--------------------------------------------------+
|12740    |{'car_id': 23, 'customer_id': 42, 'agency_id': 1} |
|12741    |{'car_id': 19, 'customer_id': 146, 'agency_id': 2}|
|12742    |{'car_id': 24, 'customer_id': 143, 'agency_id': 3}|
|12743    |{'car_id': 22, 'customer_id': 90, 'agency_id': 4} |
|12744    |{'car_id': 9, 'customer_id': 115, 'agency_id': 3} |
+---------+--------------------------------------------------+
only showing top 5 rows



In [9]:
rental_cars_df = rental_cars_df.withColumn("car_id", get_json_object(rental_cars_df.rental_info, '$.car_id')) \
                            .withColumn("customer_id", get_json_object(rental_cars_df.rental_info, '$.customer_id')) \
                            .withColumn("agency_id", get_json_object(rental_cars_df.rental_info, '$.agency_id'))

rental_cars_df.show(5, truncate=False)

+---------+--------------------------------------------------+------+-----------+---------+
|rental_id|rental_info                                       |car_id|customer_id|agency_id|
+---------+--------------------------------------------------+------+-----------+---------+
|12740    |{'car_id': 23, 'customer_id': 42, 'agency_id': 1} |23    |42         |1        |
|12741    |{'car_id': 19, 'customer_id': 146, 'agency_id': 2}|19    |146        |2        |
|12742    |{'car_id': 24, 'customer_id': 143, 'agency_id': 3}|24    |143        |3        |
|12743    |{'car_id': 22, 'customer_id': 90, 'agency_id': 4} |22    |90         |4        |
|12744    |{'car_id': 9, 'customer_id': 115, 'agency_id': 3} |9     |115        |3        |
+---------+--------------------------------------------------+------+-----------+---------+
only showing top 5 rows



In [10]:
rental_cars_df = rental_cars_df.join(cars_df, rental_cars_df.car_id == cars_df.car_id, "inner") \
                                .join(agencies_df, rental_cars_df.agency_id == agencies_df.agency_id, "inner") \
                                .join(customers_df, rental_cars_df.customer_id == customers_df.customer_id, "inner")

rental_cars_df.show(5, truncate=False)

+---------+--------------------------------------------------+------+-----------+---------+------+-----------------------------------------------------------------------------------+-------------------------------+--------+---------+---------------------------------------------------+-------------+-----------+------------------------------------------------------------------------+---------------+
|rental_id|rental_info                                       |car_id|customer_id|agency_id|car_id|car_info                                                                           |car_name                       |brand_id|agency_id|agency_info                                        |agency_name  |customer_id|customer_info                                                           |customer_name  |
+---------+--------------------------------------------------+------+-----------+---------+------+-----------------------------------------------------------------------------------+----------------

In [11]:
# Final result
rental_cars_df = rental_cars_df.select("rental_id", "car_name", "agency_name", "customer_name")

In [12]:
rental_cars_df.show(5, truncate=False)

+---------+-------------------------------+-------------+---------------+
|rental_id|car_name                       |agency_name  |customer_name  |
+---------+-------------------------------+-------------+---------------+
|12740    |Salazar Ltd Model 6            |NYC Rentals  |Sara Anderson  |
|12741    |Harris, Lloyd and Payne Model 4|LA Car Rental|Calvin Walker  |
|12742    |Alvarez-Davis Model 6          |Zapopan Auto |Shawn Tran     |
|12743    |Lopez and Sons Model 3         |SF Cars      |Edward Mccarthy|
|12744    |Levy Group Model 8             |Zapopan Auto |Antonio Haynes |
+---------+-------------------------------+-------------+---------------+
only showing top 5 rows



In [13]:
rental_cars_df.createOrReplaceTempView("rentals")

In [14]:
spark.sql("SELECT rental_id, customer_name FROM rentals").show(5)

+---------+---------------+
|rental_id|  customer_name|
+---------+---------------+
|    12740|  Sara Anderson|
|    12741|  Calvin Walker|
|    12742|     Shawn Tran|
|    12743|Edward Mccarthy|
|    12744| Antonio Haynes|
+---------+---------------+
only showing top 5 rows



In [15]:
spark.sql("SELECT agency_name, count(*) as rentals_count FROM rentals GROUP BY agency_name").show()

+-------------+-------------+
|  agency_name|rentals_count|
+-------------+-------------+
| Zapopan Auto|         4425|
|LA Car Rental|         4451|
|      SF Cars|         4481|
|  NYC Rentals|         4477|
+-------------+-------------+



#### Register temporal views

In [16]:
agencies_df.createOrReplaceTempView("agencies")
brands_df.createOrReplaceTempView("brands")
customers_df.createOrReplaceTempView("customers")
rental_cars_df.createOrReplaceTempView("rentals")
cars_df.createOrReplaceTempView("cars")

#### Identify the most popular car brands by the number of rentals

In [17]:
# Re create cars view
cars_df = spark.read.option("header", "true").schema(cars_schema).csv("/home/jovyan/notebooks/data/rentals_dataset/cars.csv")
cars_df.createOrReplaceTempView("cars")
cars_df = spark.sql("""
                    SELECT c.car_id, 
                           get_json_object(c.car_info, '$.car_name') AS car_name,
                           get_json_object(b.brand_info, '$.brand_name') AS brand_name
                    FROM cars c 
                    JOIN brands b
                    ON get_json_object(c.car_info, '$.brand_id') = b.brand_id
                    """)
cars_df.createOrReplaceTempView("cars")
spark.sql("SELECT * FROM cars").show(5)

+------+--------------------+-------------+
|car_id|            car_name|   brand_name|
+------+--------------------+-------------+
|     1|Tucker, Hull and ...|          BYD|
|     2| Howard-Snow Model 7|          BYD|
|     3|  Wagner LLC Model 2|          BMW|
|     4|  Campos PLC Model 8|Mercedes-Benz|
|     5|Archer-Patel Model 3|         Ford|
+------+--------------------+-------------+
only showing top 5 rows



In [18]:
# Re create rentals view
rental_cars_df = spark.read.option("header", "true").schema(rental_cars_schema).csv("/home/jovyan/notebooks/data/rentals_dataset/rentals/")
rental_cars_df.createOrReplaceTempView("rentals")
rental_cars_df = spark.sql("""
            SELECT r.rental_id,
                   c.car_name,
                   c.brand_name,
                   a.agency_name,
                   cus.customer_name
            FROM rentals r
            JOIN cars c ON get_json_object(r.rental_info, '$.car_id') = c.car_id
            JOIN agencies a ON get_json_object(r.rental_info, '$.agency_id') = a.agency_id
            JOIN customers cus ON get_json_object(r.rental_info, '$.customer_id') = cus.customer_id
          """)
rental_cars_df.createOrReplaceTempView("rentals")

spark.sql("""
          SELECT brand_name, count(*) AS rental_count FROM rentals 
          GROUP BY brand_name ORDER BY rental_count DESC LIMIT 3
          """).show()

+----------+------------+
|brand_name|rental_count|
+----------+------------+
|       BMW|        5471|
|     Honda|        4258|
|      Ford|        2473|
+----------+------------+



#### Find the top 5 customers

In [19]:
spark.sql("""
            SELECT customer_name, count(*) AS rental_count FROM rentals GROUP BY customer_name ORDER BY rental_count DESC LIMIT 5
          """).show()

+-----------------+------------+
|    customer_name|rental_count|
+-----------------+------------+
|Catherine Alvarez|         136|
|    Travis Butler|         133|
|     Corey Wilson|         132|
|      Ronald Hall|         131|
|    Cynthia White|         131|
+-----------------+------------+



#### Find which car generate the most revenue

In [20]:
spark.sql("SELECT car_name, count(*) AS rental_count FROM rentals GROUP BY car_name ORDER BY rental_count DESC LIMIT 5").show()

+--------------------+------------+
|            car_name|rental_count|
+--------------------+------------+
|Bryan, Barnes and...|         654|
|Clayton-Cook Mode...|         653|
|Summers, Barnett ...|         653|
|Harris, Lloyd and...|         649|
|Myers, Thornton a...|         643|
+--------------------+------------+



#### Persist data

In [21]:
rental_cars_df.write \
                .mode("overwrite") \
                .partitionBy("agency_name") \
                .parquet("/home/jovyan/notebooks/data/rentals_output/")

In [24]:
!du -h notebooks/data/rentals_output/

32K	notebooks/data/rentals_output/agency_name=Zapopan Auto
32K	notebooks/data/rentals_output/agency_name=NYC Rentals
32K	notebooks/data/rentals_output/agency_name=SF Cars
32K	notebooks/data/rentals_output/agency_name=LA Car Rental
132K	notebooks/data/rentals_output/


In [28]:
!ls -lah notebooks/data/rentals_output/agency_name=SF\ Cars | wc -l

5


25/03/11 02:16:22 ERROR TaskSchedulerImpl: Lost executor 1 on 172.24.0.5: worker lost: Not receiving heartbeat for 60 seconds
25/03/11 02:16:22 ERROR TaskSchedulerImpl: Lost executor 0 on 172.24.0.4: worker lost: Not receiving heartbeat for 60 seconds
25/03/11 02:57:10 WARN HeartbeatReceiver: Removing executor 3 with no recent heartbeats: 2428841 ms exceeds timeout 120000 ms
25/03/11 02:57:10 ERROR TaskSchedulerImpl: Lost executor 3 on 172.24.0.5: Executor heartbeat timed out after 2428841 ms
25/03/11 03:23:29 ERROR TaskSchedulerImpl: Lost executor 4 on 172.24.0.5: worker lost: Not receiving heartbeat for 60 seconds
25/03/11 03:23:29 ERROR TaskSchedulerImpl: Lost executor 2 on 172.24.0.4: worker lost: Not receiving heartbeat for 60 seconds
25/03/11 03:40:44 WARN HeartbeatReceiver: Removing executor 5 with no recent heartbeats: 952764 ms exceeds timeout 120000 ms
25/03/11 03:40:44 WARN HeartbeatReceiver: Removing executor 6 with no recent heartbeats: 956211 ms exceeds timeout 120000 ms


In [ ]:
# Stop the SparkContext
sc.stop()